In [0]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [0]:
cd /content/gdrive/My\ Drive/vip2

/content/gdrive/My Drive/vip2


In [0]:
import numpy as np
import cv2
import pickle
import keras

Using TensorFlow backend.


In [0]:
import os
count = 0
f = 'NWPU-RESISC12'
for i in os.listdir('NWPU-RESISC12'):
  for j in os.listdir(os.path.join(f,i)):
    for k in os.listdir(os.path.join(f,i,j)):
      count += 1

print('count:',count)

count: 8400


In [0]:
train_dir = 'NWPU-RESISC12/train'
validation_dir = 'NWPU-RESISC12/test'
image_size = 224
nTrain = 6600
nVal = 1800

In [0]:
descriptors_train = np.load('descriptors_train.npy', allow_pickle=True)
descriptors_test = np.load('descriptors_test.npy', allow_pickle=True)

In [0]:
print('descriptors_train.shape:',descriptors_train.shape)
print('descriptors_test.shape:',descriptors_test.shape)

descriptors_train.shape: (6600, 2)
descriptors_test.shape: (1800, 2)


In [0]:
descriptors_train[0][1].shape

(309, 128)

In [0]:
descriptors_train[1][1].shape

(637, 128)

In [0]:
from scipy.cluster.vq import kmeans, vq

with open('codebook.pkl','rb') as handle:
    codebook = pickle.load(handle)
def build_histogram(des):
    k = 50
    code, distortion = vq(des, codebook)
    bowhist,bin_edge = np.histogram(code, k, density=True)
    return bowhist

In [0]:
temp = []
for i in descriptors_train[:,1]:
  temp.append(build_histogram(i))

In [0]:
len(temp)

6600

In [0]:
des_train = np.array(temp)
des_train.shape

(6600, 50)

In [0]:
temp2 = []
for i in descriptors_test[:,1]:
  temp2.append(build_histogram(i))
des_test = np.array(temp2)
des_test.shape

(1800, 50)

In [0]:
import pickle
with open('VGG_Places_Bottleneck_Train_Features.pkl','rb') as handle:
    train_features = pickle.load(handle)
with open('VGG_Places_Bottleneck_Train_Labels.pkl','rb') as handle:
    train_labels = pickle.load(handle)
with open('VGG_Places_Bottleneck_Test_Features.pkl','rb') as handle:
    test_features = pickle.load(handle)
with open('VGG_Places_Bottleneck_Test_Labels.pkl','rb') as handle:
    test_labels = pickle.load(handle)

In [0]:
print('train_features.shape:',train_features.shape)
print('train_labels.shape:', train_labels.shape)
print('test_features.shape:',test_features.shape)
print('test_labels.shape:',test_labels.shape)

train_features.shape: (6600, 25088)
train_labels.shape: (6600, 12)
test_features.shape: (1800, 25088)
test_labels.shape: (1800, 12)


In [0]:
bottle_train = train_features
bottle_test = test_features

In [0]:
bottle_train.shape

(6600, 25088)

In [0]:
# aggregate train
agg_train_features = np.hstack((des_train,bottle_train))
agg_train_features.shape

(6600, 25138)

In [0]:
# aggregate test
agg_validation_features = np.hstack((des_test,bottle_test))
agg_validation_features.shape

(1800, 25138)

In [0]:
# aggregate label
agg_train_labels = descriptors_train[:,0].astype('int32')
agg_validation_labels = descriptors_test[:,0].astype('int32')

In [0]:
tttt = np.argmax(train_labels,axis=1)
tttt.shape

(6600,)

In [0]:
tttt

array([ 0,  0,  0, ..., 11, 11, 11])

In [0]:
count = 0
for i in range(6600):
  if tttt[i] == d_label[i]:
    count += 1
print(count)

6600


In [0]:
from keras import models
from keras import layers
from keras import optimizers
 
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=agg_train_features.shape[1]))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(12, activation='softmax'))

model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               6435584   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                3084      
Total params: 6,438,668
Trainable params: 6,438,668
Non-trainable params: 0
_________________________________________________________________


In [0]:

model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

batch_size = 20

history = model.fit(agg_train_features,
                    train_labels,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(agg_validation_features,test_labels),
                    verbose=1
                    )



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 6600 samples, validate on 1800 samples
Epoch 1/20
6600/6600 [==============================] - 10s 1ms/step - loss: 0.4651 - acc: 0.8685 - val_loss: 0.4076 - val_acc: 0.8733
Epoch 2/20
6600/6600 [==============================] - 4s 657us/step - loss: 0.1633 - acc: 0.9477 - val_loss: 0.4073 - val_acc: 0.8883
Epoch 3/20
6600/6600 [==============================] - 4s 657us/step - loss: 0.0957 - acc: 0.9682 - val_loss: 0.4172 - val_acc: 0.8922
Epoch 4/20
6600/6600 [==============================] - 4s 652us/step - loss: 0.0657 - acc: 0.9811 - val_loss: 0.4391 - val_acc: 0.8928
Epoch 5/20
6600/6600 [==============================] - 4s 653us/step - loss: 0.0483 - acc: 0.9864 - val_loss: 0.4678 - val_acc: 0.8922
Epoch 6/20
6600/6600 [==============================] - 4s 667us/step - loss: 0.0304 - acc: 0.9920 - val_loss: 0.4953 - val_acc: 0.8944
Epoch 7/20
6600/6600 [===================

In [0]:
model2 = models.Sequential()
model2.add(layers.Dense(1024, activation='relu', input_dim=agg_train_features.shape[1]))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(12, activation='softmax'))

model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1024)              25742336  
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 12)                12300     
Total params: 25,754,636
Trainable params: 25,754,636
Non-trainable params: 0
_________________________________________________________________


In [0]:

model2.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

batch_size = 20
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history = model2.fit(agg_train_features,
                    train_labels,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(agg_validation_features,test_labels),
                    callbacks=[es_callback],
                    verbose=1
                    )

Train on 6600 samples, validate on 1800 samples
Epoch 1/20
6600/6600 [==============================] - 12s 2ms/step - loss: 0.3487 - acc: 0.8924 - val_loss: 0.3834 - val_acc: 0.8889
Epoch 2/20
6600/6600 [==============================] - 11s 2ms/step - loss: 0.0880 - acc: 0.9702 - val_loss: 0.4466 - val_acc: 0.8878
Epoch 3/20
6600/6600 [==============================] - 11s 2ms/step - loss: 0.0405 - acc: 0.9862 - val_loss: 0.4899 - val_acc: 0.8944
Epoch 4/20
6600/6600 [==============================] - 11s 2ms/step - loss: 0.0206 - acc: 0.9941 - val_loss: 0.5292 - val_acc: 0.8978
Epoch 5/20
6600/6600 [==============================] - 11s 2ms/step - loss: 0.0132 - acc: 0.9965 - val_loss: 0.5797 - val_acc: 0.8994
Epoch 6/20
6600/6600 [==============================] - 11s 2ms/step - loss: 0.0057 - acc: 0.9992 - val_loss: 0.6470 - val_acc: 0.8928


In [0]:
predictions = model2.predict(agg_train_features,batch_size=batch_size)
predicted_classes = np.argmax(predictions,axis=1)

In [0]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

def scores(y_true,y_pred):
  accuracy = accuracy_score(y_true,y_pred)
  precision = precision_score(y_true,y_pred,average='macro')
  recall = recall_score(y_true,y_pred,average='macro')
  f1 = f1_score(y_true,y_pred,average='macro')
  confusionmatrix = confusion_matrix(y_true, y_pred)
  print("Accuracy:  {}".format(accuracy))
  print("Precision: {}".format(precision))
  print("Recall:    {}".format(recall))
  print("F1:        {}".format(f1))
  print("Confusion matrix:\n{}".format(confusionmatrix))

In [0]:
agg_train_labels

array([ 0,  0,  0, ..., 11, 11, 11], dtype=int32)

In [0]:
predicted_classes

array([ 0,  0,  0, ..., 11, 11, 11])

In [0]:
print('train')
scores(agg_train_labels,predicted_classes)

train
Accuracy:  0.9993939393939394
Precision: 0.9993966802646074
Recall:    0.9993939393939394
F1:        0.9993935204007492
Confusion matrix:
[[550   0   0   0   0   0   0   0   0   0   0   0]
 [  0 550   0   0   0   0   0   0   0   0   0   0]
 [  0   0 550   0   0   0   0   0   0   0   0   0]
 [  0   0   0 550   0   0   0   0   0   0   0   0]
 [  0   0   0   0 550   0   0   0   0   0   0   0]
 [  0   0   0   0   0 550   0   0   0   0   0   0]
 [  0   0   0   0   0   0 550   0   0   0   0   0]
 [  0   0   0   0   0   0   0 550   0   0   0   0]
 [  0   0   0   0   0   0   0   0 550   0   0   0]
 [  0   0   0   0   0   0   0   0   0 550   0   0]
 [  0   0   0   0   0   0   0   0   0   0 550   0]
 [  0   0   0   0   3   0   0   1   0   0   0 546]]


In [0]:
predictions = model2.predict(agg_validation_features,batch_size=batch_size)
predicted_classes = np.argmax(predictions,axis=1)
print('test')
scores(agg_validation_labels,predicted_classes)

test
Accuracy:  0.8927777777777778
Precision: 0.8943055156608372
Recall:    0.8927777777777779
F1:        0.8930488415352555
Confusion matrix:
[[137   0   1   1   1   0   5   0   2   0   3   0]
 [  1 138   0   0   0   9   0   0   1   1   0   0]
 [  1   0 138   0   1   0   0   0   9   0   1   0]
 [  1   0   0 137   1   0   1   0   1   3   5   1]
 [  0   0   0   0 144   0   0   1   1   1   2   1]
 [  0   4   0   2   0 142   0   0   0   2   0   0]
 [  3   0   0   0   1   0 139   1   0   0   5   1]
 [  2   0   0   0   0   0   0 133   2   0   8   5]
 [  0   0   5   1   0   0   0   1 130   0   6   7]
 [  1   2   1   0   4   4   0   0   3 127   3   5]
 [  1   0   0   6   1   0   1   5   4   2 124   6]
 [  4   1   0   0   6   0   0  13   4   0   4 118]]


In [0]:
# SVM
from sklearn import svm
clf = svm.SVC(gamma='scale', decision_function_shape='ovo')
clf.fit(agg_train_features, agg_train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
predictions = clf.predict([agg_validation_features])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-44-b7d4f83b2788>", line 1, in <module>
    predictions = clf.predict([agg_validation_features])
  File "/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py", line 574, in predict
    y = super().predict(X)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py", line 322, in predict
    X = self._validate_for_predict(X)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py", line 454, in _validate_for_predict
    accept_large_sparse=False)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py", line 539, in check_array
    % (array.ndim, estimator_name))
ValueError: Found array with dim 3. Estimator expected <= 2.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  F

ValueError: ignored